# `WHEN Frequency for Activity AU Description`

### `Created/Modified By: Z Liu`
 
* `1, Rewrite architecture of the code`
* `2, Fix a warning "a value is trying to be set on a copy"; using df.at[] or df.loc[] instead of df[][]`
* `3, Modify code so as to capture adhoc & ad hoc` 
* `4, Add similarity attribute to compare predictions and ground truth`
* `5, Modify code so as to capture WHEN containing conjunction such as annual and quarterly `
* `6, Remove stop words on predictions and ground truth before comparing similarity`
* `7, Modify code so as to capture WHEN containing ... as ... basis (monthly as needed basis)`
* `8, Be able to capture bi-..., bi ..., bi... etc`
* `9, Solve error, "float is not subscriptable", desc = str(desc)`
* `10, Add confidence, starting index, ending index`

In [1]:
%%time

import os
import re
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import torch.nn.functional as F

#Deep Learning Libraries
import spacy
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel, AutoModelForQuestionAnswering,pipeline

import nltk
# nltk.data.path.append('/home/mca_fix/share/nltk_data/') 
nltk.data.path.append('/home/mcafixmlpython/lib/nltk_data/') 
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()


# nlp = spacy.load("/home/mca_fix/share/en_core_web_md-3.0.0/en_core_web_md/en_core_web_md-3.0.0")
nlp = spacy.load("/home/mcafixmlpython/lib/en_core_web_md-3.0.0/en_core_web_md/en_core_web_md-3.0.0")

# Loading BERT-BASE-NLI-MEAN-TOKENS for similarity

# simi_PATH = '/home/mca_fix/share/bert-base-nli-mean-tokens/'
simi_PATH = '/home/mcafixmlpython/lib/bert-base-nli-mean-tokens/'

simi_tokenizer=AutoTokenizer.from_pretrained(simi_PATH)
simi_model=AutoModel.from_pretrained(simi_PATH,local_files_only=True)

import warnings
warnings.filterwarnings('ignore')

/home/mcafixmlpython/.local/lib/python3.6/site-packages/spacy/util.py:833: UserWarning: [W095] Model 'en_core_web_md' (3.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.2.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
Some weights of the model checkpoint at /home/mcafixmlpython/lib/bert-base-nli-mean-tokens/ were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a

CPU times: user 8.87 s, sys: 2.13 s, total: 11 s
Wall time: 10 s


In [2]:
%%time

def similar(sent):
    tokens={'input_ids':[],'attention_mask':[]}
    for sentence in sent:
        new_tokens=simi_tokenizer.encode_plus(sentence,max_length=128,truncation=True,padding='max_length',return_tensors='pt')
        tokens['input_ids'].append(new_tokens['input_ids'][0])
        tokens['attention_mask'].append(new_tokens['attention_mask'][0])
    #reformat list of tensors into single tensor
    tokens['input_ids']=torch.stack(tokens['input_ids'])
    tokens['attention_mask']=torch.stack(tokens['attention_mask'])
    #processing tokens
    outputs=simi_model(**tokens)
    outputs.keys()
    embeddings=outputs.last_hidden_state
    attention_mask=tokens['attention_mask']
    mask=attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
    masked_embeddings=embeddings * mask
    summed = torch.sum(masked_embeddings,1)
    summed_mask=torch.clamp(mask.sum(1),min=1e-9)
    mean_pooled=summed/summed_mask
    from sklearn.metrics.pairwise import cosine_similarity
    mean_pooled=mean_pooled.detach().numpy()
    x=cosine_similarity([mean_pooled[0]],mean_pooled[1:])
    return x[0][0]

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 9.06 µs


In [3]:
from nltk.corpus import stopwords

def remove_stopwords(input_text):
    #print("in remove_stopwords\n",input_text)
       
    stopwords_list = stopwords.words('english')
    newStopWords = ['citi']
    stopwords_list.extend(newStopWords)
        
    # Some words which might indicate a certain sentiment are kept via a whitelist
    #whitelist = ["n't", "not", "no"]
      
    whitelist = ["n't"]
    words = input_text.split() 
    clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 2]          
    return " ".join(clean_words)

### `20220113_ARCMs_sp.xlsx`

In [4]:
# %%time

# df = pd.read_excel('20220113_ARCMs_sp.xlsx', sheet_name="AU-ARCM Details", engine='openpyxl')

# # df = df.applymap(str)

# print(df.shape)
# df.head()

### `1000 ARCMs.xlsx`

In [5]:
%%time 

df = pd.read_excel(io='1000 ARCMs.xlsx', sheet_name="1000 ARCMs", engine='openpyxl')

# df = df[df["Activity Description contains > 20 words"] == "Pass"]
df = df[["Activity Instance Id (for AU)", "Activity AU Description", "Frequency of the activity"]]
df.reset_index(drop=True, inplace=True)

df.dropna(axis=0, inplace=True)

# # coerce type conversion to every element of a DataFrame
# df = df.applymap(str)

# text preprocessing
# df['Activity AU Description'] = df['Activity AU Description'].str.strip().str.lower()
# df['Activity AU Description'] = df['Activity AU Description'].str.replace('[^\w\s]','') # semi-annual -> semiannual

print(df.shape)
df.head()

(1000, 3)
CPU times: user 2.61 s, sys: 53.9 ms, total: 2.66 s
Wall time: 2.66 s


,Activity Instance Id (for AU),Activity AU Description,Frequency of the activity
0,709382.0,Model Development Data Sourcing: On a daily (e...,daily basis
1,709382.0,Model Development Data Sourcing: On a daily (e...,daily basis
2,709382.0,Model Development Data Sourcing: On a daily (e...,daily basis
3,709382.0,Model Development Data Sourcing: On a daily (e...,daily basis
4,709383.0,Model Development Data Sourcing confidential P...,daily basis


In [6]:
when_frequency_lst = ["daily", "weekly", 
#                       "biweekly", "realtime", "event driven", # *** from control#
                      "monthly", "quarterly", "yearly", "semiannual", "semiannually",
                      "annual", # *** from activity# "ongoing", "needed", 
                      "annually", 
                      # outliers # 
                      "each month", "every month", "calendar month",  
                      # "needed basis", "required basis",
                      "ad hoc", "adhoc"]

from typing import Tuple

def activity_when(desc: str) -> Tuple[str, str, int, int]:
    """
    
    
    """
    
    # Text preprocessing
    desc = str(desc)
#     desc = re.sub('[^\w\s]', '', desc.strip().lower())
    desc = desc.lower()
    doc = nlp(desc)
    
    activity_when_answer = None
    
    
    ########################### semi/bi, semi-annually, bi-weekly... ###########################
    semi_bi = ["semi", "bi"]
    for word in when_frequency_lst:
        
        for semi_bi_word in semi_bi:
            
            if f"""{semi_bi_word}-{word}""" in desc:
        
                activity_when_answer = f"""{semi_bi_word}-{word}"""
        
    
    
    ########################### annual and quarterly basis; adhoc or weekly basis... ########################### 
    for token in doc:
        
        # Exclude last three tokens 
        try:
        
            if (token.text in when_frequency_lst) and (doc[token.i+1].pos_ == "CCONJ") and (doc[token.i+2].text in when_frequency_lst) and (doc[token.i+3].text in ["basis", "cadence"]):
                
                span = " ".join([doc[token.i].text, doc[token.i+1].text, doc[token.i+2].text, doc[token.i+3].text])

                if any(word in when_frequency_lst for word in span.split()) and (activity_when_answer is None):
  
                    activity_when_answer = span
        
        except:
            
            pass
    
    
    
    ########################### on ... basis... ########################### 
    for token in doc:
        
        # Exclude the last token 
        try:
            
            if (token.text in when_frequency_lst) and (doc[token.i+1].text in ["basis", "cadence"]):
                
                # ... as ... basis (monthly as needed basis)
                if doc[token.i-2].text in when_frequency_lst:
                    
                    span = " ".join([doc[token.i-2].text, doc[token.i-1].text, token.text, doc[token.i+1].text])
                
                # on a/an ... basis ..
                elif (doc[token.i-2].text == "on") and ((doc[token.i-1].text == "a") or (doc[token.i-1].text == "an")):
                    
                    span = " ".join([doc[token.i-2].text, doc[token.i-1].text, token.text, doc[token.i+1].text])
                
                # on ... basis ...
                elif doc[token.i-1].text == "on":
                    
                    span = " ".join([doc[token.i-1].text, token.text, doc[token.i+1].text])
                
                else:
                    
                    span = " ".join([token.text, doc[token.i+1].text])

                if any(word in when_frequency_lst for word in span.split()) and (activity_when_answer is None):

                        activity_when_answer = span
        
        except:
            
            pass
            
              
    ########################### monthly, quarterly, annually, semi-annually... ###########################       
    for token in doc:
        
        # Exclude the last token 
        try:
        
            if ((token.dep_ == "amod") and (token.pos_ == "ADJ")) or ((token.dep_ == "advmod") and (token.pos_ == "ADV")):
                
                span = token.text

                if any(word in when_frequency_lst for word in span.split()) and (activity_when_answer is None):

                    activity_when_answer = span
        
        except:
            
            pass
                
                
                
    ########################### other outliers... ###########################
    if activity_when_answer is None:
    
        for word in when_frequency_lst:
            
            if word in desc:
                
                activity_when_answer = word
    
    
    
    ########################### confidence ###########################
    if activity_when_answer is None:
        
        confidence = "Low"
        
    else:
        
        confidence = "High"
        
        
        
    ########################### starting index, ending index ###########################
    if activity_when_answer is None:
        
        start, end = None, None
    
    else:
        
        start = desc.find(activity_when_answer)
        
        if start == -1:
            
            start, end = None, None
        
        else:
            
            end = start + len(activity_when_answer)
        
    
    
    return activity_when_answer, confidence, start, end

In [7]:
%%time

# df["when_ans_freq"] = df["Activity AU Description"].apply(func=activity_when)
df[["when_ans_freq", "confidence", "start", "end"]] = df["Activity AU Description"].\
    apply(func=lambda row: activity_when(row)[0:4]).to_list()
df["when_ans_freq"] = df["when_ans_freq"].apply(func=lambda row : "missing" if row is None else row)
df["found"] = (~df["when_ans_freq"].isna())
df['review_priority'] = np.where((df['found'] == False), "High Priority Review because of Missing When Frequency", "No Need to Review")      
df['missing_or_not'] = np.where((df['found'] == False), "Missing", "Not Missing")
df["Frequency of the activity"] = df["Frequency of the activity"].str.strip().str.lower()
df["when_ans_freq"] = df["when_ans_freq"].str.strip().str.lower()
df["similarity"] = df.apply(func=lambda row: similar(sent=[row["Frequency of the activity"], row["when_ans_freq"]]), 
                            axis=1)

CPU times: user 26min 46s, sys: 1min 40s, total: 28min 26s
Wall time: 7min 26s
